In [1]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
dict_rxn_to_gene["METLEUex"]

['6520.1', '8140.1']

In [2]:
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/edges_node_names_explainer_subgraphs.csv").drop("Unnamed: 0",axis=1)


edges_node_names_explainer_subgraphs.query("node1 == 'METLEUex'")

,node1,node2,0,1,2,3,4,5,6,7,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
28,METLEUex,met_L_c,0.008394,0.022902,0.059915,0.013377,0.010321,0.010480,0.013449,0.012713,...,0.013265,0.012531,0.071803,0.015064,0.017608,0.065341,0.012956,0.086679,0.082125,0.012253
29,METLEUex,leu_L_e,0.007510,0.038252,0.018577,0.013014,0.013034,0.009017,0.013369,0.011798,...,0.011606,0.012870,0.115802,0.022706,0.056238,0.092282,0.012522,0.121240,0.070444,0.012086
30,METLEUex,met_L_e,0.007889,0.040656,0.058529,0.013046,0.008466,0.009951,0.013248,0.012341,...,0.013127,0.012986,0.116571,0.020228,0.017969,0.057998,0.012647,0.100189,0.122094,0.012846
31,METLEUex,leu_L_c,0.013967,0.043658,0.079389,0.013706,0.011382,0.013649,0.012609,0.011909,...,0.012602,0.010712,0.057551,0.015931,0.032136,0.020654,0.014534,0.113145,0.059251,0.012503


In [3]:

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == edges_node_names_explainer_subgraphs.shape[0]


2022-10-23 13:02:10,835	INFO worker.py:1518 -- Started a local Ray instance.


In [4]:
results_for_R = pd.concat([genes, edges_node_names_explainer_subgraphs], axis=1)

results_for_R.query("node1 == 'METLEUex'")

,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
28,"[6520.1, 8140.1]",METLEUex,met_L_c,0.008394,0.022902,0.059915,0.013377,0.010321,0.010480,0.013449,...,0.013265,0.012531,0.071803,0.015064,0.017608,0.065341,0.012956,0.086679,0.082125,0.012253
29,"[6520.1, 8140.1]",METLEUex,leu_L_e,0.007510,0.038252,0.018577,0.013014,0.013034,0.009017,0.013369,...,0.011606,0.012870,0.115802,0.022706,0.056238,0.092282,0.012522,0.121240,0.070444,0.012086
30,"[6520.1, 8140.1]",METLEUex,met_L_e,0.007889,0.040656,0.058529,0.013046,0.008466,0.009951,0.013248,...,0.013127,0.012986,0.116571,0.020228,0.017969,0.057998,0.012647,0.100189,0.122094,0.012846
31,"[6520.1, 8140.1]",METLEUex,leu_L_c,0.013967,0.043658,0.079389,0.013706,0.011382,0.013649,0.012609,...,0.012602,0.010712,0.057551,0.015931,0.032136,0.020654,0.014534,0.113145,0.059251,0.012503


In [13]:
results_for_R.to_csv("./results/data/results_for_R.csv")